# First Config

## add bin path to zshrc

In [ ]:
import os           
path= os.path.join(os.getcwd(), "bin")
%run src/add_path2shrc.py $path  

## pip

In [ ]:
# create pip.conf file
import os
pip_dir=os.path.join(os.getcwd(),"user",".pip")
# create_pip_conf(pip_dir)
pip_cfg_content="[global]\nindex-url = https://mirrors.ustc.edu.cn/pypi/web/simple\ntimeout = 120\ntrusted-host = mirrors.ustc.edu.cn\n\n[install]\ntrusted-host = mirrors.ustc.edu.cn"
!echo -e "{pip_cfg_content}" > "{pip_dir}/pip.conf"

In [ ]:
# Create a link to ~/.pip
source_dir=os.path.join(os.getcwd(), 'user', '.pip')
target_dir=os.path.expanduser("~/.pip")
## Using the class
%run src/create_link.py $source_dir $target_dir --create-source --overwrite

## ssh

In [ ]:
%%bash
email="19157521820@163.com"
password="lkf.ssh.mima3"
ssh_dir="user/.ssh"
mkdir -p $ssh_dir
if [ ! -f "$ssh_dir/id_ed25519" ]; then
    ssh-keygen -t ed25519 -C "$email" -f "$ssh_dir/id_ed25519" -N "$password"
fi

In [ ]:
source_dir=os.path.join(os.getcwd(), 'user', '.ssh')
target_dir=os.path.expanduser('~/.ssh')
## Using the class
%run src/create_link.py $source_dir $target_dir --create-source --overwrite

## fonts

In [ ]:
source_dir=os.path.join(os.getcwd(), 'user', '.fonts')
target_dir="~/.fonts"
## Using the class
%run src/create_link.py $source_dir $target_dir --create-source --overwrite

## pacman-mirrors

In [ ]:
%%bash
##!/bin/bash
## 开始配置china镜像源
sudo pacman-mirrors --fasttrack 5
## 刷新包数据库
sudo pacman -Syy
## 检查并添加 archlinuxcn 源
sudo sh -c 'grep -q "archlinuxcn" /etc/pacman.conf || (echo -e "\n[archlinuxcn]\nSigLevel = Optional TrustAll\nServer = https://mirrors.ustc.edu.cn/archlinuxcn/\$arch" >> /etc/pacman.conf && echo "archlinuxcn 镜像源添加成功" || echo "archlinuxcn 镜像源添加失败")'

## archlinuxcn-keyring

In [ ]:
## 6. Pacman密钥
!sudo pacman -Sy --needed --noconfirm archlinuxcn-keyring && sudo pacman-key --init && sudo pacman-key --populate manjaro archlinux archlinuxcn && sudo pacman -Syyu --noconfirm && sudo pacman -Syyu --noconfirm

## fcitx5

In [ ]:
## 7. 输入法
!sudo pacman -Sy --needed --noconfirm manjaro-asian-input-support-fcitx5 fcitx5 fcitx5-configtool fcitx5-chinese-addons && echo -e "export GTK_IM_MODULE=fcitx\nexport QT_IM_MODULE=fcitx\nexport XMODIFIERS=@im=fcitx" > ~/.xprofile

## 软件包安装

In [ ]:
%%bash
## 定义软件包列表变量
BASE_PACKAGES="base-devel yay git-lfs haveged"
MEDIA_PACKAGES="vlc ffmpeg qbittorrent"
FONT_PACKAGES="adobe-source-han-sans-otc-fonts adobe-source-han-serif-otc-fonts wqy-zenhei noto-fonts-cjk unzip-natspec"
APP_PACKAGES="chromium telegram-desktop  partitionmanager clash-verge-rev aria2  docker docker-compose obs-studio onlyoffice-desktopeditors"

## 安装函数
install_packages() {
    local category=$1
    local packages=$2
    
    echo "安装 $category: $packages"
    sudo pacman -Sy --needed --noconfirm $packages
}

## 安装所有软件包
install_packages "基础工具" "$BASE_PACKAGES"
install_packages "多媒体工具" "$MEDIA_PACKAGES" 
install_packages "字体包" "$FONT_PACKAGES"
install_packages "应用程序" "$APP_PACKAGES"

sudo systemctl enable haveged


In [ ]:
%%bash
AUR_PACKAGES="google-chrom visual-studio-code-bin"
install_yay_packages() {
    local category=$1
    local packages=$2
    
    echo "安装 $category: $packages"
    yay -Sy --needed --noconfirm $packages
}
install_yay_packages "AUR软件包" "$AUR_PACKAGES"

## add brtfs part to fstab

In [ ]:
# %%bash
# echo "Adding /data partition"
# lsblk -f
# sudo mkdir -p /data && echo "UUID=180161ab-546d-4dfe-969e-e1cf9938195e /data btrfs defaults,compress=zstd,noatime 0 0" | sudo tee -a /etc/fstab

# systemctl daemon-reload
# sudo mount -a
# df -h | grep /data
# sudo chown $USER:$USER /data
# sudo chmod 755 /data


## system config
### dolphin
- 详情
- 隐藏文件
- 终端
- 右键vscode打开



#### 用户目录中英文切换


In [ ]:
%%bash
# 切换为英文用户目录
LANG=en_US.UTF-8 xdg-user-dirs-update --force
# 切换为中文用户目录
# LANG=zh_CN.UTF-8 xdg-user-dirs-update --force

In [ ]:
import os
from pathlib import Path

## Chinese to English directory name mapping
CHINESE_TO_ENGLISH = {
    "视频": "Videos",
    "图片": "Pictures", 
    "文档": "Documents",
    "下载": "Downloads",
    "音乐": "Music",
    "模板": "Templates",
    "公共": "Public",
    "桌面": "Desktop",
}


def conver_dirname_to_english(base_path):
    """
    Convert Chinese directory names to English standard names
    
    Args:
        base_path (str): Base path containing Chinese directories
    """
    base_path = Path(base_path)
    
    for chinese_name, english_name in CHINESE_TO_ENGLISH.items():
        chinese_dir = base_path / chinese_name
        english_dir = base_path / english_name
        if chinese_dir.exists() and not english_dir.exists():
            chinese_dir.rename(english_dir)
            print(f"✅ Renamed: {chinese_name} -> {english_name}")
        elif chinese_dir.exists():
            print(f"⚠️  Both {chinese_name} and {english_name} exist, skipping")
        elif english_dir.exists():
            print(f"✅ English directory already exists: {english_dir}")


if __name__ == "__main__":

    path=os.path.join(os.getcwd(),"user")
    # 创建英文目录
    for english_name in CHINESE_TO_ENGLISH.values():
        english_dir = os.path.join(path, english_name)
        if not os.path.exists(english_dir):
            os.makedirs(english_dir, exist_ok=True)
            print(f"📁 Created: {english_dir}")
            
    conver_dirname_to_english(path)

    links= [(os.path.join(path, name), os.path.expanduser(f"~/{name}")) for name in CHINESE_TO_ENGLISH.values()]
    from src.create_link import LinkManager
    LinkManager.bulk_create_links(links)


#### 位置面板配置

In [ ]:
# 位置面板配置文件地址 ～/.local/share/user-places.xbel
import os
import xml.etree.ElementTree as ET

CHINESE_TO_ENGLISH = {
    "视频": "Videos",
    "图片": "Pictures", 
    "文档": "Documents",
    "下载": "Downloads",
    "音乐": "Music",
    "模板": "Templates",
    "公共": "Public",
    "桌面": "Desktop",
}
def get_config_path(custom_path=None):
    if custom_path:
        return os.path.expanduser(custom_path)
    return os.path.expanduser("~/.local/share/user-places.xbel")


def find_target_elements(root,target_titles=None):

    found_elements = []
    
    for bookmark in root.findall('.//bookmark'):
        title_elem = bookmark.find('title')
        if title_elem is not None and title_elem.text in target_titles:
            found_elements.append((bookmark, title_elem))
    
    return found_elements
def convert_title_to_english(root,target_titles=None):
    """将中文标题转换为英文"""
    if target_titles is None:
        return root
    found_elements = find_target_elements(root,target_titles)
    for bookmark, title_elem in found_elements:
        if title_elem.text in list(CHINESE_TO_ENGLISH.keys()):
            print(f"  Converted {title_elem.text} to {CHINESE_TO_ENGLISH[title_elem.text]}")
            title_elem.text = CHINESE_TO_ENGLISH[title_elem.text]
    return root

def update_href_by_title(root, title, new_path):
    """修改指定title的bookmark元素的href属性"""
    for bookmark in root.findall('.//bookmark'):
        title_elem = bookmark.find('title')
        if title_elem is not None and title_elem.text == title:
            # 确保路径是绝对路径
            abs_path = os.path.abspath(os.path.expanduser(new_path))
            bookmark.set('href', f"file://{abs_path}")
            return True
    return False

if __name__ == "__main__":
    # 加载配置文件
    config_path = get_config_path()
    # 解析XML树
    tree = ET.parse(config_path)
    root = tree.getroot()

    # 将中文标题转换为英文
    chinese_titles = list(CHINESE_TO_ENGLISH.keys())
    conver_dirname_to_english = convert_title_to_english(root,chinese_titles)
    # 定义新的路径映射 (title: new_path)
    title_to_new_path = {
        CHINESE_TO_ENGLISH[title]: f"~/{CHINESE_TO_ENGLISH[title]}"
        for title in CHINESE_TO_ENGLISH.keys()
    }
    print("Updating href attributes...")
    for title, new_path in title_to_new_path.items():
        update_href_by_title(conver_dirname_to_english, title, new_path)
        print(f"  Updated {title} to {new_path}")

    tree.write(config_path)
    print("XML file updated successfully.")

### yakuake

In [ ]:
%%bash
cat > ~/.config/yakuakerc << 'EOF'
[Animation]
Frames=11  ; 动画帧数，设置为11帧

[Appearance]
TerminalHighlightOnManualActivation=true
Translucency=true

[AutoOpen]
PollMouse=true  ; 启用鼠标轮询，自动检测鼠标活动

[Dialogs]
FirstRun=false

[Window]
DynamicTabTitles=true  ; 启用动态标签页标题
Height=23
KeepOpen=false
KeepOpenAfterLastSessionCloses=true  ; 在最后一个会话关闭后保持窗口打开
ShowSystrayIcon=false
Width=80

EOF

# Backup

## btrfs创建系统快照教程

In [ ]:
%%bash
# 创建快照
# 检查分区的子卷
echo "检查根分区的子卷"
sudo btrfs subvolume list /
# 为分区创建快照目录
sudo mkdir -p /.snapshots
echo "创建快照"
# 为分区创只读快照（系统文件）
TIMESTAMP=$(date +%Y%m%d_%H%M%S)
sudo btrfs subvolume snapshot -r / /.snapshots/${TIMESTAMP}_root
# /home
# sudo btrfs subvolume snapshot -r /home /home/.snapshots/${TIMESTAMP}_home
# /data
# sudo btrfs subvolume snapshot -r /data /data/.snapshots/${TIMESTAMP}_data

In [ ]:
%%bash
# 删除快照
sudo btrfs subvolume list /
# 保留最新的快照
# sudo btrfs subvolume delete /.snapshots/root_


In [ ]:
%%bash
# 查看所有快照
echo "=== 根分区快照 ==="
sudo btrfs subvolume list / | grep snapshots
echo "=== 家目录分区快照 ==="
sudo btrfs subvolume list /home | grep snapshots
echo "=== 数据分区快照 ==="
sudo btrfs subvolume list /data | grep snapshots

### 从快照恢复的完整流程

In [ ]:
%%bash
#   从 Live USB 启动并挂载所有分区
# 创建挂载点
sudo mkdir -p /mnt/{root,home,data}

# 挂载根分区
sudo mount /dev/nvme0n1p1 /mnt/root

# 挂载家目录分区
sudo mount /dev/nvme0n1p3 /mnt/home

# 挂载数据分区（可选）
sudo mount /dev/nvme0n1p5 /mnt/data

cd /mnt/root
# 删除损坏的子卷
sudo btrfs subvolume delete @
sudo btrfs subvolume delete @cache  
sudo btrfs subvolume delete @log
# 从快照恢复
sudo btrfs subvolume snapshot .snapshots/root_20241027_153000_ro @
sudo btrfs subvolume snapshot .snapshots/cache_20241027_153000_ro @cache
sudo btrfs subvolume snapshot .snapshots/log_20241027_153000_ro @log



# 恢复家目录分区
cd /mnt/home

# 先检查家目录的子卷名称
sudo btrfs subvolume list .

# 假设子卷名是 @home，删除并恢复
sudo btrfs subvolume delete @home
sudo btrfs subvolume snapshot .snapshots/home_20241027_153000_ro @home

# 恢复数据分区（可选）
cd /mnt/data

# 先检查数据目录的子卷名称
sudo btrfs subvolume list .

# 假设子卷名是 @data，删除并恢复
sudo btrfs subvolume delete @data
sudo btrfs subvolume snapshot .snapshots/data_20241027_153000_ro @data

# 卸载所有分区
cd /
sudo umount /mnt/{root,home,data}
sudo rmdir /mnt/{root,home,data}

# 重启系统
sudo reboot

## Git 配置




In [ ]:
%%bash
## 安装 Git
sudo pacman -Sy git --needed --noconfirm

git config --global user.email "19157521820@163.com" && git config --global user.name "kefu"

## add ssh key to github
## 登录 GitHub

## 点击右上角头像 → Settings

## 左侧菜单选择 SSH and GPG keys

## 点击 New SSH key

## 标题（如 "My Laptop"），粘贴复制的公钥

## 点击 Add SSH key

## 测试 SSH 连接
ssh -T git@github.com

## 设置 Git 拒绝大文件提交


In [ ]:
!git config --global http.postBuffer 209715200
!git config --global core.compression 9

In [ ]:
%%bash
git init
git remote add origin git@github.com:luckykefu/ManjaroUsingLog.git


In [ ]:
%%bash
git add .
git commit -m "create src folder"
git push


## [clash sub link](https://zhuzhuzhu.whtjdasha.com/api/v1/client/subscribe?token=7ff879517322f152fcc639c592bc59b9)



# Software config


## cryptomator


In [ ]:
%%bash
# keepassdb 
advise rub clean goodbye sketch terrain maximum recycle hi report theme viable length wonderful consent roughly remind overly slice available candidate sincere satisfy shrink yet elite elite secretary contract matching choir implement number through secondary ashamed pulse nonsense widen so ancestor pain chemistry ritual

# kepasskey
strive stamp lab delighted silent potential interfere obesity issue eastern situated detail recommend municipal sweet casual chocolate pose integrate support tear notion normally current react nonsense sit pride fresh tender hall excited soil intimate cannot convince slowly sail least grab minister shy grief poverty


## keepassxc更好用

## proxychains


In [ ]:
%%bash
sudo pacman -Sy --needed --noconfirm proxychains-ng

## 删除所有现有的 socks 代理配置
sudo sed -i '/^socks[45]/d' /etc/proxychains.conf


echo "socks5 192.168.0.101 7897" | sudo tee -a /etc/proxychains.conf

## virtualbox


In [ ]:
%%bash

uname -r
sudo pacman -Sy virtualbox linux612-virtualbox-host-modules virtualbox-ext-vnc --noconfirm --needed

## 加载内核模块
sudo modprobe vboxdrv
sudo modprobe vboxnetadp
sudo modprobe vboxnetflt

## 添加用户到 vboxusers 组
sudo usermod -aG vboxusers $USER

newgrp vboxusers

## 重启生效
## reboot


##  Davinci Resolve

In [ ]:
## 1. 访问官网下载页面
## 打开浏览器访问: https://www.blackmagicdesign.com/products/davinciresolve/

## 2. 手动下载Linux版本
## 下载完成后，文件通常在 ~/Downloads/ 目录

## 运行.run文件


## aria2 webui
https://github.com/mayswind/AriaNg/releases

In [ ]:
import os
from pathlib import Path

def setup_aria2_config():
    """Setup aria2 configuration directory and files"""
    
    ## Create config directory
    config_dir = Path.home() / ".config" / "aria2"
    config_dir.mkdir(parents=True, exist_ok=True)
    print(f"📁 Created config directory: {config_dir}")
    
    ## Create session file
    session_file = config_dir / "aria2.session"
    session_file.touch()
    print(f"📄 Created session file: {session_file}")
    
    ## Create configuration file
    config_content = """## 基本设置
dir=${HOME}/Downloads
continue=true
max-connection-per-server=16
min-split-size=1M
split=16

## 进度保存
input-file=${HOME}/.config/aria2/aria2.session
save-session=${HOME}/.config/aria2/aria2.session
save-session-interval=60

## RPC 设置 (用于 WebUI)
enable-rpc=true
rpc-listen-all=true
rpc-listen-port=6800
rpc-secret=mysecret

## BT 设置
bt-enable-lpd=true
bt-max-peers=50
bt-request-peer-speed-limit=1M
bt-stop-timeout=0

## 性能优化
max-overall-download-limit=0
max-overall-upload-limit=1K
seed-ratio=1.0
seed-time=0

## 日志
log=${HOME}/.config/aria2/aria2.log
log-level=warn
"""
    
    config_file = config_dir / "aria2.conf"
    with open(config_file, 'w', encoding='utf-8') as f:
        f.write(config_content)
    print(f"⚙️  Created config file: {config_file}")
    
    print("✅ aria2 configuration completed")

## Run the function
setup_aria2_config()

In [ ]:
import os
from pathlib import Path
import subprocess

def setup_aria2_service():
    """Setup aria2 as a systemd user service"""
    
    ## Create systemd user directory
    service_dir = Path.home() / ".config" / "systemd" / "user"
    service_dir.mkdir(parents=True, exist_ok=True)
    print(f"📁 Created service directory: {service_dir}")
    
    ## Create service file content
    service_content = """[Unit]
Description=Aria2c Download Manager
After=network.target

[Service]
Type=forking
ExecStart=/usr/bin/aria2c --conf-path=%h/.config/aria2/aria2.conf -D
ExecStop=/usr/bin/killall aria2c
Restart=on-failure

[Install]
WantedBy=default.target
"""
    
    ## Write service file
    service_file = service_dir / "aria2.service"
    with open(service_file, 'w', encoding='utf-8') as f:
        f.write(service_content)
    print(f"⚙️  Created service file: {service_file}")
    
    ## Reload systemd and enable service
    try:
        subprocess.run(["systemctl", "--user", "daemon-reload"], check=True)
        print("✅ Systemd daemon reloaded")
        
        subprocess.run(["systemctl", "--user", "enable", "aria2"], check=True)
        print("✅ Service enabled")
        
        subprocess.run(["systemctl", "--user", "start", "aria2"], check=True)
        print("✅ Service started")
        
        print("🎉 aria2 system service installed successfully")
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to setup service: {e}")
        return False
    
    return True

## Run the function
setup_aria2_service()

In [ ]:
%%bash
## 不使用 jq，通过 grep 和 sed 提取版本号
LATEST_TAG=$(curl -s "https://api.github.com/repos/mayswind/AriaNg/releases/latest" | grep '"tag_name":' | sed -E 's/.*"([^"]+)".*/\1/')
echo "最新版本: $LATEST_TAG"

wget "https://github.com/mayswind/AriaNg/releases/download/${LATEST_TAG}/AriaNg-${LATEST_TAG}-AllInOne.zip"

echo "✅ 下载完成"

## unzip aria2 webui
unzip -o AriaNg-${LATEST_TAG}-AllInOne.zip -d ./bin/AriaNg

rm -f "AriaNg-${LATEST_TAG}-AllInOne.zip"

python3 -m http.server 8080 --directory ./bin/AriaNg


- 打开浏览器访问 http://localhost:8080/
- AriaNg 设置
- RPC
- Aria2 RPC 协议
- websocket
- 密钥

## ollama


In [ ]:
%%bash
## install ollama dir
install_dir=$(pwd)
## 创建安装目录
mkdir -p $install_dir/ollama
## 创建模型存储目录
mkdir -p $install_dir/ollama/models
## GitHub 会自动重定向到最新版本
wget https://github.com/ollama/ollama/releases/latest/download/ollama-linux-amd64.tgz -O $install_dir/ollama-linux-amd64.tgz
## 解压主程序
tar -xzf $install_dir/ollama-linux-amd64.tgz -C $install_dir/ollama/
rm -f $install_dir/ollama-linux-amd64.tgz
## 添加到 PATH
## 使用变量
OLLAMA_PATH="export PATH=\"$install_dir/ollama/bin:\$PATH\""
OLLAMA_MODELS_ENV="export OLLAMA_MODELS=\"$install_dir/ollama/models\""

grep -qxF "$OLLAMA_PATH" ~/.zshrc || echo "$OLLAMA_PATH" >> ~/.zshrc
grep -qxF "$OLLAMA_MODELS_ENV" ~/.zshrc || echo "$OLLAMA_MODELS_ENV" >> ~/.zshrc
source ~/.zshrc
## restart the app



In [ ]:
## AMD 显卡驱动 安装

In [ ]:
%%bash
## ## 安装 ollama ROCm lib
## wget https://github.com/ollama/ollama/releases/latest/download/ollama-linux-amd64-rocm.tgz -O ollama-linux-amd64-rocm.tgz

## 解压 ROCm 库到 Ollama 目录
tar -xzf ollama-linux-amd64-rocm.tgz -C ollama/



## AMD 显卡驱动

In [ ]:
%%bash

## 更新系统
echo "正在更新系统..."
sudo pacman -Syu --noconfirm
## 安装必要的开发工具
echo "安装开发工具..."
sudo pacman -Sy --needed --noconfirm base-devel git wget curl cmake

## 安装ROCm驱动和开发工具包
echo "安装ROCm驱动..."
sudo pacman -Sy --needed --noconfirm rocm-hip-sdk rocm-opencl-sdk rocm-smi-lib

## 添加用户到必要的硬件访问组
echo "配置用户组权限..."
sudo usermod -a -G render,video,kvm $USER

## 验证安装
echo "验证ROCm安装..."
/opt/rocm/bin/rocminfo


## 将 ROCm 的 bin 目录添加到 PATH 环境变量（如果尚未添加）
echo "将 ROCm 的 bin 目录添加到 PATH 环境变量（如果尚未添加）"
grep -q ':/opt/rocm/bin' <<< $PATH || echo 'export PATH=$PATH:/opt/rocm/bin:/opt/rocm/opencl/bin' >> ~/.zshrc
grep -q ':/opt/rocm/lib' <<< $LD_LIBRARY_PATH || echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/rocm/lib:/opt/rocm/lib64' >> ~/.zshrc

echo "安装完成！请重启系统使所有配置生效。"
echo "重启后使用以下命令激活环境:"
echo "source ~/rocm-env/bin/activate"



## shadowsocks

In [ ]:
%%bash
##!bash
## 服务器端manjaro版本
sudo pacman -Sy shadowsocks --noconfirm --needed
## 设置配置变量
SERVER="0.0.0.0"
PORT=8388
PASSWORD="vpntest"
METHOD="aes-256-gcm"
TIMEOUT=300

## 创建配置文件
sudo mkdir -p /etc/shadowsocks
sudo tee /etc/shadowsocks/config.json > /dev/null << EOF
{
    "server": "$SERVER",
    "server_port": $PORT,
    "password": "$PASSWORD",
    "method": "$METHOD",
    "timeout": $TIMEOUT,
    "fast_open": false,
    "mode": "tcp_and_udp"
}
EOF

## 验证配置
sudo cat /etc/shadowsocks/config.json
sudo systemctl start shadowsocks-server@config
sudo systemctl status shadowsocks-server@config

## 添加 iptables 规则开放8388端口
sudo iptables -A INPUT -p tcp --dport 8388 -j ACCEPT
sudo iptables -A INPUT -p udp --dport 8388 -j ACCEPT

## 保存 iptables 规则（根据系统选择合适的方法）
sudo iptables-save | sudo tee /etc/iptables/iptables.rules

## 检查规则是否添加成功
sudo iptables -L INPUT -n | grep 8388

## 成功


In [ ]:
%%bash
## 客户端
sudo pacman -Sy shadowsocks --noconfirm --needed
SERVER="207.148.77.242"
PASSWORD="vpntest"
LOCAL_PORT=1080

sudo tee /etc/shadowsocks/config.json > /dev/null << EOF
{
    "server": "$SERVER",
    "server_port": 8388,
    "local_address": "127.0.0.1",
    "local_port": "$LOCAL_PORT",
    "password": "$PASSWORD",
    "method": "aes-256-gcm",
    "timeout": 300
}
EOF
cat /etc/shadowsocks/config.json
sudo systemctl start shadowsocks@config 
sudo systemctl status shadowsocks@config   
sudo systemctl stop shadowsocks@config  

curl --socks5 127.0.0.1:1080 ipinfo.io/ip

In [ ]:
## shadowsocks

## 服务器端 ubuntu版本
sudo apt update
sudo apt install shadowsocks-libev
sudo nano /etc/shadowsocks-libev/config.json
{
    "server": "0.0.0.0",
    "server_port": 8388,
    "password": "your_password_here",
    "method": "aes-256-gcm",
    "timeout": 300,
    "fast_open": false
}

## 启动 Shadowsocks 服务
sudo systemctl start shadowsocks-libev

## 设置开机自动启动
sudo systemctl enable shadowsocks-libev

## 检查服务状态，确认是否正常运行
sudo systemctl status shadowsocks-libev
## 开放 Shadowsocks 服务端口
sudo ufw allow 8388/tcp

## 重新加载防火墙规则
sudo ufw reload

## 检查防火墙状态，确认端口已开放
sudo ufw status

## 检查8388端口是否在监听
netstat -tlnp | grep 8388

## 或者使用 ss 命令
ss -tlnp | grep 8388


In [ ]:
%%bash
## ssh: connect to host 207.148.77.242 port 22: Connection refused
ssh-keygen -R 207.148.77.242 

## clash-verge

In [ ]:

import yaml
from pathlib import Path

def create_clash_config_cn(config_path="~/.config/clash/config.yaml"):
    """
    创建中文注释的 Clash 配置文件
    """
    config = {
        ## HTTP(S) 代理服务端口
        'port': 7890,
        
        ## SOCKS5 代理服务端口  
        'socks-port': 7891,
        
        ## 设置为 true 以允许来自其他 LAN IP 地址的连接
        'allow-lan': False,
        
        ## Clash 路由工作模式
        ## rule: 基于规则的数据包路由
        ## global: 所有数据包将被转发到单个节点  
        ## direct: 直接将数据包转发到互联网
        'mode': 'rule',
        
        ## 日志级别: info / warning / error / debug / silent
        'log-level': 'info',
        
        ## RESTful Web API 监听地址
        'external-controller': '127.0.0.1:9090',
        
        ## 代理服务器配置
        'proxies': [],
        
        ## 代理组配置
        'proxy-groups': [
            {
                'name': '🚀 代理',
                'type': 'select',
                'proxies': ['DIRECT']
            }
        ],
        
        ## 规则配置
        'rules': [
            ## 本地网络直连
            'DOMAIN-SUFFIX,local,DIRECT',
            'IP-CIDR,127.0.0.0/8,DIRECT', 
            'IP-CIDR,10.0.0.0/8,DIRECT',
            'IP-CIDR,172.16.0.0/12,DIRECT',
            'IP-CIDR,192.168.0.0/16,DIRECT',
            
            ## 国内网站直连
            'GEOIP,CN,DIRECT',
            
            ## 常用国外网站走代理
            'DOMAIN-SUFFIX,google.com,🚀 代理',
            'DOMAIN-SUFFIX,gstatic.com,🚀 代理', 
            'DOMAIN-SUFFIX,googleapis.com,🚀 代理',
            'DOMAIN-SUFFIX,github.com,🚀 代理',
            'DOMAIN-SUFFIX,github.io,🚀 代理',
            'DOMAIN-SUFFIX,githubassets.com,🚀 代理',
            'DOMAIN-SUFFIX,gitlab.com,🚀 代理',
            'DOMAIN-SUFFIX,youtube.com,🚀 代理',
            'DOMAIN-SUFFIX,ytimg.com,🚀 代理',
            'DOMAIN-SUFFIX,twitter.com,🚀 代理',
            'DOMAIN-SUFFIX,facebook.com,🚀 代理',
            'DOMAIN-SUFFIX,instagram.com,🚀 代理',
            'DOMAIN-SUFFIX,whatsapp.com,🚀 代理',
            'DOMAIN-SUFFIX,telegram.org,🚀 代理',
            'DOMAIN-SUFFIX,reddit.com,🚀 代理',
            'DOMAIN-SUFFIX,wikipedia.org,🚀 代理',
            'DOMAIN-SUFFIX,stackoverflow.com,🚀 代理',
            'DOMAIN-SUFFIX,stackexchange.com,🚀 代理',
            'DOMAIN-SUFFIX,medium.com,🚀 代理',
            'DOMAIN-SUFFIX,dropbox.com,🚀 代理',
            'DOMAIN-SUFFIX,notion.so,🚀 代理',
            'DOMAIN-SUFFIX,figma.com,🚀 代理',
            'DOMAIN-SUFFIX,discord.com,🚀 代理',
            'DOMAIN-SUFFIX,slack.com,🚀 代理',
            'DOMAIN-SUFFIX,zoom.us,🚀 代理',
            
            ## 广告屏蔽
            'DOMAIN-KEYWORD,adservice,REJECT',
            'DOMAIN-SUFFIX,doubleclick.net,REJECT',
            'DOMAIN-SUFFIX,googlesyndication.com,REJECT',
            
            ## 最终规则
            'MATCH,🚀 代理'
        ]
    }
    
    config_path = Path(config_path).expanduser()
    config_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(config_path, 'w', encoding='utf-8') as f:
        yaml.dump(config, f, allow_unicode=True, default_flow_style=False, sort_keys=False)
    
    print(f"✅ Clash 配置文件已创建: {config_path}")
    print("📝 现在你可以在 'proxies' 部分添加你的代理服务器")
    print("📍 HTTP 代理: 127.0.0.1:7890")
    print("📍 SOCKS5 代理: 127.0.0.1:7891")
    print("🌐 控制面板: http://127.0.0.1:9090")
    
    return config_path

## 创建配置文件
create_clash_config_cn()

In [ ]:
def add_proxies_to_config(config_path, proxies):
    """
    Add proxies to existing Clash config file
    
    Args:
        config_path: Path to config file
        proxies: List of proxy configurations
    """
    config_path = Path(config_path).expanduser()
    
    if not config_path.exists():
        print("❌ Config file not found, creating new one...")
        create_clash_config_cn(config_path)
    
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    
    ## Add new proxies
    config['proxies'].extend(proxies)
    
    ## Update proxy groups with new proxies
    proxy_names = [p['name'] for p in proxies if p['type'] not in ['direct', 'reject']]
    
    for group in config['proxy-groups']:
        if group['name'] not in ['DIRECT', 'REJECT']:
            ## Add new proxies to existing groups (avoid duplicates)
            for proxy_name in proxy_names:
                if proxy_name not in group['proxies']:
                    group['proxies'].append(proxy_name)
    
    with open(config_path, 'w', encoding='utf-8') as f:
        yaml.dump(config, f, allow_unicode=True, default_flow_style=False, sort_keys=False)
    
    print(f"✅ Added {len(proxies)} proxies to config")
    return config_path

## Example usage to add proxies later:
"""
## Define your proxies
my_proxies = [
    {
        'name': 'My-SS-Server',
        'type': 'ss',
        'server': 'example.com',
        'port': 8388,
        'cipher': 'chacha20-ietf-poly1305',
        'password': 'mypassword',
        'udp': True
    }
]

## Add them to config
add_proxies_to_config('~/.config/clash/config.yaml', my_proxies)
"""
myproxies=my_proxies = [
    {
        'name': 'My-SS-Server',
        'type': 'ss',
        'server': 'example.com',
        'port': 8388,
        'cipher': 'chacha20-ietf-poly1305',
        'password': 'mypassword',
        'udp': True
    }
]
## Add them to config
add_proxies_to_config('~/.config/clash/config.yaml', my_proxies)

In [ ]:
## 终端启动代理
export ALL_PROXY="socks5://192.168.0.101:7897"

## google chrome



In [ ]:
## 使用代理启动谷歌浏览器
!google-chrome-stable  --proxy-server="socks5://192.168.0.101:7897"   

#### floccus Sync bookmarks

#### Proxy SwitchyOmega 3


## yabridge

In [ ]:


```bash
sudo pacman -Sy --needed --noconfirm yabridge yabridgectl wine wine-gecko wine-mono  winetricks 

## Acon Digital
WINEPREFIX=/run/media/kf/WD/wine/vst/AconDigital winecfg
yabridgectl add /run/media/kf/WD/wine/vst/AconDigital/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## Antares
No success
WINEPREFIX=/run/media/kf/WD/wine/vst/AntaresAutoTune winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/AntaresAutoTune wine Antares\ Auto-Tune\ Pro\ v11.0.0.exe 
yabridgectl add /run/media/kf/WD/wine/vst/AntaresAutoTune/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/AntaresDeEsser wine 
yabridgectl add /run/media/kf/WD/wine/vst/AntaresDeEsser/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## Arturia
WINEPREFIX=/run/media/kf/WD/wine/vst/Arturia winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Arturia wine 
yabridgectl add /run/media/kf/WD/wine/vst/Arturia/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## BABY Audio
WINEPREFIX=/run/media/kf/WD/wine/vst/BABYAudio winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/BABYAudio winecfg
yabridgectl add /run/media/kf/WD/wine/vst/BABYAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

##BBE  
WINEPREFIX=/run/media/kf/WD/wine/vst/BBESonicSweet winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/BBESonicSweet wine s
yabridgectl add /run/media/kf/WD/wine/vst/BBESonicSweet/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/BBEStompBoard  winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/BBEStompBoard wine s
yabridgectl add /run/media/kf/WD/wine/vst/BBEStompBoard/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## Boz Digital Labs
WINEPREFIX=/run/media/kf/WD/wine/vst/BozDigitalLabs wine 
yabridgectl add /run/media/kf/WD/wine/vst/BozDigitalLabs/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## CableGuys
WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysShaperBox winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysShaperBox wine 
yabridgectl add /run/media/kf/WD/wine/vst/CableGuysShaperBox/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysKickStart winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysKickStart wine 
yabridgectl add /run/media/kf/WD/wine/vst/CableGuysKickStart/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## CamelCrusher
WINEPREFIX=/run/media/kf/WD/wine/vst/CamelCrusher wine 
yabridgectl add /run/media/kf/WD/wine/vst/CamelCrusher/drive_c/Program\ Files/VSTPlugins/
yabridgectl sync --prune

## Celemony
WINEPREFIX=/run/media/kf/WD/wine/vst/Celemony winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Celemony wine 
yabridgectl add /run/media/kf/WD/wine/vst/Celemony/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## cradle
WINEPREFIX=/run/media/kf/WD/wine/vst/Cradle winetricks dxvk 
WINEPREFIX=/run/media/kf/WD/wine/vst/Cradle wine 
yabridgectl add /run/media/kf/WD/wine/vst/Cradle/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## d16grouprepeater
WINEPREFIX=/run/media/kf/WD/wine/vst/D16GroupRepeater winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/D16GroupRepeater wine s
yabridgectl add /run/media/kf/WD/wine/vst/D16GroupRepeater/drive_c/Program\ Files//Steinberg/VSTPlugins/
yabridgectl sync --prune

##dadalife
no success

## deniseaudio
WINEPREFIX=/run/media/kf/WD/wine/vst/DeniseAudio winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/DeniseAudio wine 
yabridgectl add /run/media/kf/WD/wine/vst/DeniseAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## exciteaudio
WINEPREFIX=/run/media/kf/WD/wine/vst/ExciteAudioHarmonic winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/ExciteAudioHarmonic wine s
yabridgectl add /run/media/kf/WD/wine/vst/ExciteAudioHarmonic/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune


## fabfilter
WINEPREFIX=/run/media/kf/WD/wine/vst/FabFilter winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/FabFilter wine s
yabridgectl add /run/media/kf/WD/wine/vst/FabFilter/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## goodhertz
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz wine 
yabridgectl add /run/media/kf/WD/wine/vst/Goodhertz/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz wine cmd /c 
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz wine regedit 

## Harman Audio
WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconLXP winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconLXP wine 
yabridgectl add /run/media/kf/WD/wine/vst/LexiconLXP/drive_c/Program\ Files/Steinberg/VSTPlugins/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconPCM winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconPCM wine 
yabridgectl add /run/media/kf/WD/wine/vst/LexiconPCM/drive_c/Program\ Files/Steinberg/VSTPlugins/
yabridgectl sync --prune

## Heckmann Audio - u-he Diva v1.4.8
WINEPREFIX=/run/media/kf/WD/wine/vst/uheDiva wine 
yabridgectl add /run/media/kf/WD/wine/vst/uheDiva/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## ikmultimedia
WINEPREFIX=/run/media/kf/WD/wine/vst/IKMultimediaTracks6 winetricks mfc42 dxvk  
WINEPREFIX=/run/media/kf/WD/wine/vst/IKMultimediaTracks6 wine 
yabridgectl add /run/media/kf/WD/wine/vst/IKMultimediaTracks6/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## izotope
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeAurora winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeAurora wine 
yabridgectl add /run/media/kf/WD/wine/vst/IzotopeAurora/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeRX11 winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeRX11 wine 
yabridgectl add /run/media/kf/WD/wine/vst/IzotopeRX11/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune
##############################
## black screen
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOzone11 winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOzone11 wine 
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOnectar winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOnectar wine
##############################

## MinimalAudio
WINEPREFIX=/run/media/kf/WD/wine/vst/MinimalAudio winetricks  dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/MinimalAudio wine 
yabridgectl add /run/media/kf/WD/wine/vst/MinimalAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## Native Instruments
WINEPREFIX=/run/media/kf/WD/wine/vst/NativeInstruments winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/NativeInstruments wine *.exe
yabridgectl add /run/media/kf/WD/wine/vst/NativeInstruments/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## NembriniAudio

WINEPREFIX=/run/media/kf/WD/wine/vst/NembriniAudio winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/NembriniAudio wine 
yabridgectl add /run/media/kf/WD/wine/vst/NembriniAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## okesound
WINEPREFIX=/run/media/kf/WD/wine/vst/Okesound winecfg
yabridgectl add /run/media/kf/WD/wine/vst/Okesound/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## overloud 
WINEPREFIX=/run/media/kf/WD/wine/vst/Overloud winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/Overloud wine 
yabridgectl add /run/media/kf/WD/wine/vst/Overloud/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## pluginalliance
WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceBPE winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceBPE wine s
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceBPE/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceFAS wine 
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceFAS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceTTS winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceTTS wine 
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceTTS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceCopy winecfg
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceCopy/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## PolyverseWider
WINEPREFIX=/run/media/kf/WD/wine/vst/PolyverseWider winetricks dxbk
WINEPREFIX=/run/media/kf/WD/wine/vst/PolyverseWider wine 
yabridgectl add /run/media/kf/WD/wine/vst/PolyverseWider/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## pulsar
no success
## RDGA
WINEPREFIX=/run/media/kf/WD/wine/vst/RDGA wine 
yabridgectl add /run/media/kf/WD/wine/vst/RDGA/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## slatedigital
WINEPREFIX=/run/media/kf/WD/wine/vst/Slatedigital winecfg
yabridgectl add /run/media/kf/WD/wine/vst/Slatedigital/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/Slatedigitalana2 winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Slatedigitalana2 wine 
yabridgectl add /run/media/kf/WD/wine/vst/Slatedigitalana2/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## softube
no success

## ssl
WINEPREFIX=/run/media/kf/WD/wine/vst/SSL winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/SSL winecfg
yabridgectl add /run/media/kf/WD/wine/vst/SSL/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## soundtoys
no success

## sonible
WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleDeEss winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleDeEss wine 
yabridgectl add /run/media/kf/WD/wine/vst/SonibleDeEss/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleSmartEQ4 winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleSmartEQ4 wine 
yabridgectl add /run/media/kf/WD/wine/vst/SonibleSmartEQ4/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/Sonible winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Sonible wine 
yabridgectl add /run/media/kf/WD/wine/vst/Sonible/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## Soundtoys
WINEPREFIX=/run/media/kf/WD/wine/vst/Soundtoys wine 
yabridgectl add /run/media/kf/WD/wine/vst/Soundtoys/drive_c/Program\ Files\ \(x86\)/VstPlugins/
yabridgectl sync --prune

## SpectrasonicsOmnishpere
WINEPREFIX=/run/media/kf/WD/wine/vst/SpectrasonicsOmnisphere winetricks  dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/SpectrasonicsOmnisphere wine 
yabridgectl add /run/media/kf/WD/wine/vst/SpectrasonicsOmnisphere/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## TDeEsser
WINEPREFIX=/run/media/kf/WD/wine/vst/TDeEsser wine 
yabridgectl add /run/media/kf/WD/wine/vst/TDeEsser/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## unitedplugin
WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginFS wine 
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginFS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginIA wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginIA/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginJMGS wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginJMGS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginMA wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginMA/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginSD wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginSD/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## ValhallaDSP
WINEPREFIX=/run/media/kf/WD/wine/vst/ValhallaDSP wine 
yabridgectl add /run/media/kf/WD/wine/vst/ValhallaDSP/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## voxengo
WINEPREFIX=/run/media/kf/WD/wine/vst/Voxengo winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/Voxengo wine 
yabridgectl add /run/media/kf/WD/wine/vst/Voxengo/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## waves
no success

## WAKSHMRReverb
WINEPREFIX=/run/media/kf/WD/wine/vst/WAKSHMRReverb wine 
yabridgectl add /run/media/kf/WD/wine/vst/WAKSHMRReverb/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## wavesfactory
WINEPREFIX=/run/media/kf/WD/wine/vst/WavesFactory winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/WavesFactory wine 
yabridgectl add /run/media/kf/WD/wine/vst/WavesFactory/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## XLNRC20
WINEPREFIX=/run/media/kf/WD/wine/vst/XLNRC20 winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/XLNRC20 wine 
yabridgectl add /run/media/kf/WD/wine/vst/XLNRC20/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## youlean 
WINEPREFIX=/run/media/kf/WD/wine/vst/Youlean wine 
yabridgectl add /run/media/kf/WD/wine/vst/Youlean/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## zynaptiaa
No success
WINEPREFIX=/run/media/kf/WD/wine/vst/Zynaptia winetricks mfc42 vcrun2022
WINEPREFIX=/run/media/kf/WD/wine/vst/Zynaptia wine 
yabridgectl add /run/media/kf/WD/wine/vst/Zynaptia/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## 添加 vst3 插件
yabridgectl add path/to/vst3/plugin
yabridgec tl sync --prune

```

#### wine


In [ ]:

## GTK+工具
sudo pacman -Sy --needed --noconfirm zenity  
sudo pacman -Sy --needed --noconfirm gnutls lib32-gnutls lib32-alsa-lib lib32-pulseaudio

## xstudio
WINEPREFIX=/run/media/kf/sdb/Linux/exe/XStudio winetricks dxvk cjkfonts vcrun2022
WINEPREFIX=/run/media/kf/sdb/Linux/exe/XStudio wine  
## wechat
WINEPREFIX=/run/media/kf/sdb/Linux/exe/WeChat winetricks dxvk vcrun2022  cjkfonts
WINEPREFIX=/run/media/kf/sdb/Linux/exe/WeChat wine 


## 邮件


In [ ]:

19157521820@163.com

18285998868@163.com

3124568493@qq.com

kefu1820@gmail.com


## Account

- +86 19157521820
- google chrome kefu1820@gmail.com

# Uninstall

In [ ]:
%%bash
sudo pacman -Rns code --noconfirm

# delete the folder
folder_path=$(pwd)/bin/vscode
rm -rf $folder_path
# delete the pth in ~/.zshrc
sed -i '/^export.*vscode/d' ~/.zshrc

